<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/Feature_Selection_and_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install tsfresh
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '15myxitNBue9FKPBvNm-j_FS_fiElz3C8' # files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 1000 files
Processing folder: 100 files
Processing folder: 10 files


In [42]:
def read_file(filename):
  return pd.read_csv(filename, sep=',')

extracted_train_feature = pd.DataFrame(read_file('extracted-100hz-train-feature-20percent-1000files.csv'))
extracted_test_feature = pd.DataFrame(read_file('extracted-100hz-test-feature-20percent-1000files.csv'))
extracted_train_target = pd.DataFrame(read_file('extracted-100hz-train-target-20percent-1000files.csv'))
extracted_test_target = pd.DataFrame(read_file('extracted-100hz-test-target-20percent-1000files.csv'))

In [43]:
extracted_train_target = extracted_train_target.drop(columns=['Unnamed: 0'])
extracted_test_target = extracted_test_target.drop(columns=['Unnamed: 0'])
extracted_train_feature = extracted_train_feature.drop(columns=['Unnamed: 0'])
extracted_test_feature = extracted_test_feature.drop(columns=['Unnamed: 0'])

In [44]:
extracted_train_feature.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,1.0,1.0,1.0,250.81,101.4769,0.038292,0.000415,0.000000,0.37,...,1.280506,1.919827,3.842121,1.386197,2.285083,3.157043,3.966588,4.670473,0.0,35.13
1,0.0,0.0,0.0,1.0,250.05,102.0841,0.042379,0.000389,0.000047,0.37,...,1.123708,1.765862,3.701585,1.398722,2.307939,3.216146,4.018596,4.716842,0.0,35.13
2,0.0,0.0,1.0,1.0,231.03,94.2721,0.041493,0.000520,-0.000067,0.37,...,1.092491,1.741591,3.702245,1.364469,2.287314,3.155369,3.939051,4.591458,0.0,35.13
3,0.0,0.0,1.0,1.0,219.25,89.8207,0.042157,0.000481,-0.000054,0.37,...,1.346895,2.015920,3.921954,1.462090,2.406550,3.360859,4.188281,4.904547,0.0,35.13
4,0.0,1.0,1.0,1.0,239.70,98.4812,0.040408,0.000408,-0.000033,0.37,...,1.158979,1.819628,3.760082,1.356556,2.270489,3.194101,4.025864,4.789695,0.0,35.13


In [45]:
selected_features_train = select_features(extracted_train_feature, extracted_train_target['color'])

In [46]:
selected_features_test = extracted_test_feature[selected_features_train.columns]

In [52]:
selected_features_train

,Temp(°C)__quantile__q_0.7,Temp(°C)__quantile__q_0.8,Temp(°C)__quantile__q_0.6,Temp(°C)__median,Temp(°C)__quantile__q_0.9,Temp(°C)__quantile__q_0.4,Temp(°C)__quantile__q_0.1,Temp(°C)__minimum,Temp(°C)__quantile__q_0.3,Temp(°C)__mean_n_absolute_max__number_of_maxima_7,...,V_Out(V)__sample_entropy,Temp(°C)__partial_autocorrelation__lag_3,"Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.2","I_Out(A)__fft_coefficient__attr_""imag""__coeff_98","I_In__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2",Temp(°C)__autocorrelation__lag_3,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_7","I_Out(A)__fft_coefficient__attr_""abs""__coeff_74",Temp(°C)__number_peaks__n_3
0,35.13,35.13,34.87,34.87,35.13,34.87,34.87,34.62,34.87,35.13,...,0.722427,-0.021780,0.000000,0.300371,0.003327,0.000000,-0.021407,7.895804,1.788283,16.0
1,35.13,35.13,34.87,34.87,35.13,34.87,34.87,34.62,34.87,35.13,...,0.736839,-0.059312,0.000000,1.117479,0.004347,0.000000,-0.062171,8.010951,0.925069,22.0
2,35.13,35.13,34.87,34.87,35.13,34.87,34.87,34.62,34.87,35.13,...,0.763102,0.009653,0.000000,0.599488,0.004732,0.000000,0.009946,7.276172,1.518125,25.0
3,35.13,35.13,35.13,35.13,35.13,34.87,34.87,34.62,34.87,35.13,...,0.781012,-0.059296,0.033861,-1.746776,0.004674,0.130249,-0.059379,6.258480,0.864320,2.0
4,35.13,35.13,35.13,35.13,35.13,34.87,34.87,34.62,34.87,35.13,...,0.757505,0.052658,0.031109,-1.381044,0.003543,0.119654,0.053027,7.925760,1.924330,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,32.87,32.87,32.87,32.87,32.87,32.87,32.87,32.62,32.87,33.12,...,0.763507,0.050515,0.000000,-0.553587,0.004083,0.000000,0.052872,7.732517,2.564870,22.0
796,32.87,32.87,32.87,32.87,32.87,32.87,32.87,32.62,32.87,33.12,...,0.759779,-0.042735,0.000000,-1.490128,0.003744,0.000000,-0.043624,7.445567,2.486950,28.0
797,32.87,32.87,32.87,32.87,32.87,32.87,32.62,32.62,32.87,33.12,...,0.768456,0.058145,0.000000,-1.732985,0.003718,0.000000,0.054993,8.230867,0.498740,17.0
798,32.87,32.87,32.87,32.87,32.87,32.87,32.62,32.62,32.87,33.12,...,0.763484,0.035267,0.000000,0.135276,0.004484,0.000000,0.034698,4.609617,0.667991,12.0


In [51]:
selected_features_test

,Temp(°C)__quantile__q_0.7,Temp(°C)__quantile__q_0.8,Temp(°C)__quantile__q_0.6,Temp(°C)__median,Temp(°C)__quantile__q_0.9,Temp(°C)__quantile__q_0.4,Temp(°C)__quantile__q_0.1,Temp(°C)__minimum,Temp(°C)__quantile__q_0.3,Temp(°C)__mean_n_absolute_max__number_of_maxima_7,...,V_Out(V)__sample_entropy,Temp(°C)__partial_autocorrelation__lag_3,"Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.2","I_Out(A)__fft_coefficient__attr_""imag""__coeff_98","I_In__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2",Temp(°C)__autocorrelation__lag_3,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_7","I_Out(A)__fft_coefficient__attr_""abs""__coeff_74",Temp(°C)__number_peaks__n_3
0,35.38,35.38,35.38,35.38,35.38,35.38,35.38,35.13,35.38,35.63,...,0.749920,-0.104388,0.0,0.242583,0.004059,0.0,-0.104710,6.942429,1.186125,38.0
1,35.38,35.38,35.38,35.38,35.63,35.38,35.38,35.13,35.38,35.63,...,0.745092,-0.037631,0.0,1.336115,0.002334,0.0,-0.034792,6.546131,1.992054,28.0
2,35.38,35.38,35.38,35.38,35.63,35.38,35.38,35.13,35.38,35.63,...,0.733659,0.010327,0.0,-0.304916,0.004009,0.0,0.016542,7.652275,2.148368,30.0
3,35.38,35.38,35.38,35.38,35.38,35.38,35.38,35.13,35.38,35.63,...,0.750902,-0.016882,0.0,-0.227271,0.002559,0.0,-0.016654,6.909954,3.399718,35.0
4,35.38,35.38,35.38,35.38,35.63,35.38,35.38,35.13,35.38,35.63,...,0.718512,0.060310,0.0,1.638485,0.004628,0.0,0.068353,8.644359,1.278641,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,37.88,38.13,37.88,37.88,38.13,37.88,37.88,37.63,37.88,38.13,...,0.819172,-0.032539,0.0,-1.005761,0.003035,0.0,-0.033761,8.090304,1.908156,49.0
3196,37.88,38.13,37.88,37.88,38.13,37.88,37.88,37.63,37.88,38.13,...,0.793951,0.009623,0.0,-0.052470,0.003186,0.0,0.011383,7.843438,1.908023,45.0
3197,37.88,38.13,37.88,37.88,38.13,37.88,37.88,37.63,37.88,38.13,...,0.848634,0.067892,0.0,2.228091,0.003293,0.0,0.067967,6.673115,1.196895,35.0
3198,37.88,38.13,37.88,37.88,38.13,37.88,37.88,37.63,37.88,38.13,...,0.831808,-0.036872,0.0,-0.009825,0.002353,0.0,-0.034348,7.450701,1.512626,45.0


In [53]:
extracted_test_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
3195,3
3196,3
3197,3
3198,3


In [61]:
model = RandomForestClassifier()
model.fit(selected_features_train,extracted_train_target)
y_pred = model.predict(selected_features_test)
accuracy = accuracy_score(extracted_test_target, y_pred)
precision = precision_score(extracted_test_target, y_pred, average='weighted')
print(classification_report(extracted_test_target, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(confusion_matrix(extracted_test_target, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.99      0.92      0.96       800
           1       0.99      0.33      0.50       800
           2       0.47      0.87      0.61       800
           3       0.73      0.66      0.70       800

    accuracy                           0.70      3200
   macro avg       0.80      0.70      0.69      3200
weighted avg       0.80      0.70      0.69      3200

Accuracy: 0.6969
Precision: 0.7970
[[739   0  58   3]
 [  0 267 443  90]
 [  4   3 694  99]
 [  0   1 269 530]]


In [64]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

final_model = xgb.XGBClassifier(
    # objective="multi:softmax",
    # random_state=42,
    alpha=10,
    reg_lambda=1,
    # max_depth=10,
    min_child_weight=10,
    # subsample=0.8,
    colsample_bytree=0.8
)

final_model.fit(selected_features_train,extracted_train_target)
y_pred = final_model.predict(selected_features_test)
accuracy = accuracy_score(extracted_test_target, y_pred)
precision = precision_score(extracted_test_target, y_pred, average='weighted')
print(classification_report(extracted_test_target, y_pred))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(confusion_matrix(extracted_test_target, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.77      0.86       800
           1       0.61      0.69      0.64       800
           2       0.73      0.85      0.78       800
           3       0.72      0.66      0.69       800

    accuracy                           0.74      3200
   macro avg       0.76      0.74      0.74      3200
weighted avg       0.76      0.74      0.74      3200

Accuracy: 0.7406
Precision: 0.7585
[[612  73  93  22]
 [  0 550 160  90]
 [ 11  14 678  97]
 [  0 269   1 530]]
